# SageMaker Serverless Inference
## Sklearn Regression Example

Amazon SageMaker Serverless Inference is a purpose-built inference option that makes it easy for customers to deploy and scale ML models. Serverless Inference is ideal for workloads which have idle periods between traffic spurts and can tolerate cold starts. Serverless endpoints also automatically launch compute resources and scale them in and out depending on traffic, eliminating the need to choose instance types or manage scaling policies.

For this notebook we'll be working with the a custom Sklearn model to train a model and then deploy a serverless endpoint. We will be using the public S3 California housing dataset for this example.

<b>Update: </b>
SageMaker Serverless Inference is now supported by the [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-serverless-inference). This makes it very easy to deploy Serverless models that you are also training on SageMaker. As an alternative you can also use the general Boto3 Python SDK to create Serverless models, you can find an example notebook [here](https://github.com/aws/amazon-sagemaker-examples/blob/master/serverless-inference/Serverless-Inference-Walkthrough.ipynb).

<b>Notebook Setting</b>
- <b>SageMaker Classic Notebook Instance</b>: ml.m5.xlarge Notebook Instance & conda_python3 Kernel
- <b>SageMaker Studio</b>: Python 3 (Data Science)
- <b>Regions Available</b>: SageMaker Serverless Inference is currently available in the following regions: US East (Northern Virginia), US East (Ohio), US West (Oregon), EU (Ireland), Asia Pacific (Tokyo) and Asia Pacific (Sydney)

## Setup

In [ ]:
# install tree command (helpful in printing folder structures)
!apt-get install tree

# setup AWS cli
!mkdir -p ~/.aws && cp /content/drive/MyDrive/AWS/d01_admin/* ~/.aws
!chmod 600 ~/.aws/credentials
!pip install awscli

# install boto3 and sagemaker
!pip install boto3
!pip install sagemaker

# install dependencies
!pip install pyathena
!pip install awswrangler
!pip install smclarify
!pip install sagemaker-experiments
!pip install sagemaker-tensorflow
!pip install smclarify

# install nlp libs
!pip install transformers

In [1]:
# imports
import boto3
import sagemaker
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from sagemaker.estimator import Estimator

In [103]:
# global variables
role = "sagemakerRole"
base_job_prefix = "sklearn-housing"
training_instance_type = "ml.m5.xlarge"

In [3]:
# setup sagemaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role_arn = "arn:aws:iam::{}:role/{}".format(account_id, role)
sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)

## Data Ingest

In [106]:
# Retrieve the California Housing dataset from a publicly hosted S3 bucket
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz .
!tar -zxf cal_housing.tgz

# Create a dataframe from the California housing dataset that we can upload to S3
columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
    "medianHouseValue",
]
df = pd.read_csv("CaliforniaHousing/cal_housing.data", names=columns, header=None)
display(df.head())

# Splitting data in 80-20 split to use testing data for model inference later
train = df.iloc[:16000,:]
test = df.iloc[16001:,:]

# Train and test csv
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

# Upload data to S3
training_input_path = sess.upload_data('train.csv', key_prefix=base_job_prefix + '/training')
print(training_input_path)

#verify data uploaded properly
training_data = pd.read_csv(training_input_path, sep = ',')
display(training_data.head())

download: s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz to ./cal_housing.tgz


,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


s3://sagemaker-us-east-1-390354360073/sklearn-housing/training/train.csv


,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


## Model Training

Now, we train a custom model using our Sklearn training script. In this example, we also provide [inference handler functions](https://docs.aws.amazon.com/sagemaker/latest/dg/adapt-inference-container.html) to work with input and output functionality for inference, feel free to adjust this for how you want your endpoint to ingest and respond to data.

In [ ]:
!mkdir -p script

In [ ]:
%%writefile script/train.py
import argparse, os
import boto3
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import joblib
import pickle


if __name__ == '__main__':
    
    #Passing in environment variables and hyperparameters for our training script
    parser = argparse.ArgumentParser()
    
    #Hyperparamaters
    parser.add_argument('--estimators', type=int, default=15)
    
    #sm_model_dir: model artifacts stored here after training
    parser.add_argument('--sm-model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--model_dir', type=str)
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    
    args, _ = parser.parse_known_args()
    estimators     = args.estimators
    model_dir  = args.model_dir
    sm_model_dir = args.sm_model_dir
    training_dir   = args.train
    
    ############
    #Reading in data
    ############
    df = pd.read_csv(training_dir + '/train.csv',sep=',')
    
    ############
    #Preprocessing data
    ############
    X = df.drop('medianHouseValue', axis = 1)
    y = df['medianHouseValue']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train) 
    X_test = sc.transform(X_test)
    
    ###########
    #Model Building
    ###########
    regressor = RandomForestRegressor(n_estimators=estimators)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    
    ###########
    #Save the Model
    ###########
    joblib.dump(regressor, os.path.join(args.sm_model_dir, "model.joblib"))
    

###########
#Model Serving
###########
    
"""
Deserialize fitted model
"""
def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

"""
input_fn
    request_body: The body of the request sent to the model.
    request_content_type: (string) specifies the format/variable type of the request
"""
def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        request_body = json.loads(request_body)
        inpVar = request_body['Input']
        return inpVar
    else:
        raise ValueError("This model only supports application/json input")

"""
predict_fn
    input_data: returned array from input_fn above
    model (sklearn model) returned model loaded from model_fn above
"""
def predict_fn(input_data, model):
    return model.predict(input_data)

"""
output_fn
    prediction: the returned value from predict_fn above
    content_type: the content type the endpoint expects to be returned. Ex: JSON, string
"""

def output_fn(prediction, content_type):
    res = int(prediction[0])
    respJSON = {'Output': res}
    return respJSON

In [108]:
#Docs: https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html
from sagemaker.sklearn import SKLearn
sk_estimator = SKLearn(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.c5.18xlarge',
                          py_version='py3',
                          framework_version='0.23-1',
                          script_mode=True,
                          hyperparameters={
                              'estimators': 20
                          }
                         )

#Training
sk_estimator.fit({'train': training_input_path})

2022-06-10 08:54:36 Starting - Starting the training job...
2022-06-10 08:55:02 Starting - Preparing the instances for trainingProfilerReport-1654851275: InProgress
............
2022-06-10 08:57:00 Downloading - Downloading input data
2022-06-10 08:57:00 Training - Downloading the training image...
2022-06-10 08:57:31 Training - Training image download completed. Training in progress..2022-06-10 08:57:33,285 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-06-10 08:57:33,288 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-10 08:57:33,296 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-06-10 08:57:33,663 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-10 08:57:33,673 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-10 08:57:33,685 sagemaker-training-toolkit INFO     No GPUs detected (no

![](https://github.com/sparsh-ai/notebooks/raw/main/img/rec1-sklearn-housing-serverless.gif)

## Create Serverless Config Using The SageMaker SDK

For Serverless Inference you need two parameters: Memory Size and Max Concurrency. The current max concurrent invocations for a single endpoint, known as <b>MaxConcurrency</b>, can be any value from <b>1 to 50</b>, and <b>MemorySize</b> can be any of the following: <b>1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB</b>.

In [109]:
from sagemaker.serverless import ServerlessInferenceConfig
serverless_config = ServerlessInferenceConfig(memory_size_in_mb=4096, max_concurrency=3)

## Deploy Serverless Endpoint

In [110]:
import time
from time import gmtime, strftime
endpoint_name = "sklearn-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [111]:
sk_estimator.deploy(endpoint_name = endpoint_name, serverless_inference_config=serverless_config)

------!

![](https://github.com/sparsh-ai/notebooks/raw/main/img/rec2-sklearn-housing-serverless.gif)

## Inference

Let's invoke the endpoint with a sample data point from our train set.

In [112]:
#Create sample data point
import json
samp = pd.read_csv('train.csv')
samp = samp.drop('medianHouseValue', 1)
samp = samp[:1]
request_body = {"Input": samp.values.tolist()}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [113]:
import boto3
client = boto3.client('sagemaker-runtime')
content_type = "application/json"
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)
result = json.loads(response['Body'].read().decode())['Output']
result

472745